# Mesa: Agent-based Modeling Framework
From https://mesa.readthedocs.io/en/latest/, 

> Mesa is a modular framework for building, analyzing and visualizing agent-based models.

> Agent-based models are computer simulations involving multiple entities (the agents) acting and interacting with one another based on their programmed behavior. Agents can be used to represent living cells, animals, individual humans, even entire organizations or abstract entities. Sometimes, we may have an understanding of how the individual components of a system behave, and want to see what system-level behaviors and effects emerge from their interaction. Other times, we may have a good idea of how the system overall behaves, and want to figure out what individual behaviors explain it. Or we may want to see how to get agents to cooperate or compete most effectively. **Or we may just want to build a cool toy with colorful little dots moving around.**

# Modeling Module
Modules used to build the models themselves: a model and agent classes, a scheduler to determine the sequence in which the agents act, and space for them to move around on.

In [1]:
import pandas as pd
import seaborn as sns

from mesa import Agent

In [2]:
class MyAgent(Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name

    def step(self):
        print("{} activated".format(self.name))
        # Whatever else the agent does when activated

In [3]:
from mesa import Model

from mesa.time import RandomActivation
from mesa.space import MultiGrid

In [4]:
class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(10, 10, torus=True)
        
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)

    def step(self):
        self.schedule.step()

## `mesa.time`

In [5]:
# https://mesa.readthedocs.io/en/latest/apis/time.html
from mesa.time import (
    BaseScheduler,
    RandomActivation, # most commonly used
    RandomActivationByType,
    SimultaneousActivation,
    StagedActivation
)

## `mesa.space`

In [6]:
# https://mesa.readthedocs.io/en/latest/apis/space.html
from mesa.space import (
    Grid,
    SingleGrid,
    MultiGrid,
    HexGrid,
    ContinuousSpace,
    NetworkGrid
)

## Running the model

In [8]:
n_agents = 5

model = MyModel(n_agents)
model.step()

4 activated
0 activated
1 activated
2 activated
3 activated


In [9]:
model.step()

1 activated
3 activated
2 activated
4 activated
0 activated


In [10]:
model.step()

4 activated
1 activated
3 activated
0 activated
2 activated


# Analysis
Tools to collect data generated from your model, or to run it multiple times with different parameter values.

In [13]:
# https://mesa.readthedocs.io/en/latest/apis/datacollection.html
from mesa import DataCollector

In [26]:
def get_agent_count(m):
    return m.schedule.get_agent_count()


class MyModel(Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(10, 10, torus=True)
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10), self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)
        
        # ###########
        # for analysis
        dataCollectorParameters = dict(
            model_reporters={"agent_count": get_agent_count},
            agent_reporters={"name": lambda a: a.name}
        )
        
        self.datacollector = DataCollector(**dataCollectorParameters)
        # ###########
        
    def step(self):
        self.schedule.step()
        
        # for analysis
        self.datacollector.collect(self)

In [27]:
n_agents = 5

model = MyModel(n_agents)

for t in range(5):
    model.step()
    print()

4 activated
0 activated
3 activated
2 activated
1 activated

3 activated
2 activated
1 activated
4 activated
0 activated

4 activated
3 activated
0 activated
1 activated
2 activated

1 activated
2 activated
4 activated
3 activated
0 activated

1 activated
0 activated
2 activated
4 activated
3 activated



## Import data collection results into a pandas dataframe

In [28]:
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

In [29]:
model_df.shape

(5, 1)

In [30]:
 model_df

,agent_count
0,5
1,5
2,5
3,5
4,5


In [31]:
agent_df.shape

(25, 1)

In [32]:
agent_df

name
Step AgentID      
1    0           0
     1           1
     2           2
     3           3
     4           4
2    0           0
     1           1
     2           2
     3           3
     4           4
3    0           0
     1           1
     2           2
     3           3
     4           4
4    0           0
     1           1
     2           2
     3           3
     4           4
5    0           0
     1           1
     2           2
     3           3
     4           4

## Batch running for higher statistical confidence

In [33]:
# https://mesa.readthedocs.io/en/latest/apis/batchrunner.html
from mesa.batchrunner import batch_run

In [34]:
parameters = {
    "n_agents": range(1, 20)
}


run = batch_run(
    MyModel, 
    parameters, 
    max_steps=10
)

 74%|███████████████████████████████████████              | 14/19 [00:00<00:00, 97.72it/s]

0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
0 activated
1 activated
0 activated
1 activated
0 activated
1 activated
0 activated
0 activated
1 activated
1 activated
0 activated
1 activated
0 activated
0 activated
1 activated
1 activated
0 activated
1 activated
0 activated
0 activated
1 activated
0 activated
1 activated
0 activated
2 activated
1 activated
0 activated
2 activated
1 activated
0 activated
1 activated
2 activated
2 activated
0 activated
1 activated
1 activated
2 activated
0 activated
2 activated
1 activated
0 activated
1 activated
2 activated
0 activated
0 activated
2 activated
1 activated
0 activated
2 activated
1 activated
1 activated
0 activated
2 activated
1 activated
0 activated
2 activated
0 activated
1 activated
2 activated
3 activated
1 activated
0 activated
3 activated
2 activated
2 activated
3 activated
0 activated
1 activated
2 activated
0 activated
3 activated
1 activated
2 activated
3 ac

100%|█████████████████████████████████████████████████████| 19/19 [00:00<00:00, 70.41it/s]


0 activated
12 activated
11 activated
8 activated
9 activated
1 activated
14 activated
7 activated
9 activated
16 activated
10 activated
8 activated
6 activated
1 activated
0 activated
15 activated
5 activated
3 activated
4 activated
14 activated
13 activated
11 activated
12 activated
2 activated
14 activated
6 activated
2 activated
1 activated
9 activated
15 activated
7 activated
16 activated
13 activated
4 activated
0 activated
12 activated
10 activated
11 activated
8 activated
3 activated
5 activated
12 activated
1 activated
15 activated
9 activated
16 activated
11 activated
8 activated
2 activated
10 activated
6 activated
4 activated
14 activated
3 activated
7 activated
5 activated
13 activated
0 activated
5 activated
3 activated
2 activated
10 activated
11 activated
7 activated
8 activated
0 activated
15 activated
17 activated
6 activated
16 activated
13 activated
12 activated
14 activated
4 activated
1 activated
9 activated
12 activated
15 activated
16 activated
0 activated
2 ac

In [35]:
batch_df = pd.DataFrame(run)

In [36]:
batch_df.shape

(190, 7)

In [37]:
batch_df.head()

,RunId,iteration,Step,n_agents,agent_count,AgentID,name
0,0,0,10,1,1,0,0
1,1,0,10,2,2,0,0
2,1,0,10,2,2,1,1
3,2,0,10,3,3,0,0
4,2,0,10,3,3,1,1


# Resources
* https://github.com/projectmesa/mesa
* https://dadaromeo.github.io/posts/mesa-a-library-for-agent-based-modeling-in-python/
* https://mesa.readthedocs.io/en/latest/tutorials/intro_tutorial.html
* https://mesa.readthedocs.io/en/latest/tutorials/adv_tutorial.html